In [1]:
from demoparser2 import DemoParser
import pandas as pd
import os
import sys

from loaders.mapname import get_mapname


In [2]:


parse = DemoParser("13_29.dem")


In [3]:
def parse_demo(filename: str):
# parse filename
# file names are a tuple of matchid_mapid.dem
    parser = DemoParser(filename)
    filename_info = filename.split(os.sep)[-1][:-4].split('_')
    matchid = int(filename_info[0])
    mapid = int(filename_info[1])

    map_player_rounds = []
    round_count = parser.parse_event("round_freeze_end").shape[0]
    parser = DemoParser(filename)

    players = parser.parse_player_info()
    mapname = get_mapname(parser)
    player_teams = parser.parse_ticks(['team_clan_name'], ticks=[100])[['team_clan_name', 'name']]

    main_df = parser.parse_event('smokegrenade_detonate').drop(columns=['user_steamid'])
    ticks_df = parser.parse_ticks(['total_rounds_played']).drop(columns=['steamid'])
    ticks_df.rename(columns={'name': 'user_name'}, inplace=True)
    ticks_df['tick'] = ticks_df['tick'] + 640
    main_df = pd.merge(main_df, ticks_df, how='inner', on=['tick', 'user_name']).drop_duplicates(subset=['total_rounds_played', 'tick', 'user_name'])
    main_df['map_name'] = mapname
    main_df['matchid'] = matchid
    main_df['mapid'] = mapid
    main_df['player_team'] = main_df['user_name'].map(player_teams.set_index('name')['team_clan_name'])
    main_df.rename(columns={'total_rounds_played': 'round_number'}, inplace=True)
    main_df['round_number'] = main_df['round_number'] + 1

    win_df = parser.parse_event('round_end')
    win_df['round'] -= 1
    win_df.drop(columns=['reason', 'tick'], inplace=True)
    win_df.drop(index=0, inplace=True)  
    win_df.rename(columns={'round':'round_number' }, inplace=True)

    main_df = pd.merge(main_df, win_df, how='inner', on='round_number')

    df3 = parser.parse_ticks(['team_name'])
    df3.rename(columns={'name': 'user_name'}, inplace=True)
    df3.drop(columns=['steamid'], inplace=True)

    main_df = pd.merge(main_df, df3, how='inner', on=['user_name', 'tick'])
    main_df['team_name'] = main_df['team_name'].replace('TERRORIST', 'T')
    
    main_df['is_win'] = main_df['team_name'] == main_df['winner']

    end_df = parser.parse_event('round_end')
    end_df['round'] -= 1
    end_df.drop(columns=['reason', 'winner'], inplace=True)
    end_df.drop(index=0, inplace=True)  
    end_df.rename(columns={'round':'round_number', 'tick': 'end_tick'}, inplace=True)

    new_row_data = {'round_number': 0, 'end_tick': 0}
    new_row_df = pd.DataFrame([new_row_data])

    end_df = pd.concat([new_row_df, end_df], ignore_index=True)
    end_df['round_number'] += 1
    
    main_df = pd.merge(main_df, end_df, how='inner', on='round_number')
    main_df['round_tick'] = main_df['tick'] - main_df['end_tick']

    return main_df

dem = parse_demo("13_29.dem")
dem


,entityid,tick,user_name,x,y,z,round_number,map_name,matchid,mapid,player_team,winner,team_name,is_win,end_tick,round_tick
0,385,3200,yxngstxr,665.714355,401.153534,121.991577,1,de_ancient,13,29,HEROIC,CT,T,False,0,3200
1,304,3209,LNZ,1291.354004,471.039032,126.290375,1,de_ancient,13,29,HEROIC,CT,T,False,0,3209
2,371,3363,TeSeS,1232.540894,-399.975494,60.440563,1,de_ancient,13,29,Team Falcons,CT,CT,True,0,3363
3,100,8760,Magisk,-1137.333008,-877.204224,35.969467,2,de_ancient,13,29,Team Falcons,CT,CT,True,6752,2008
4,85,9463,kyxsan,1115.198608,-985.491211,5.528252,2,de_ancient,13,29,Team Falcons,CT,CT,True,6752,2711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,391,181598,kyxsan,-976.921631,-194.174942,92.031250,22,de_ancient,13,29,Team Falcons,T,T,True,178536,3062
153,84,182291,LNZ,521.729980,103.724792,154.758850,22,de_ancient,13,29,HEROIC,T,CT,False,178536,3755
154,470,183458,SunPayus,519.949646,136.839844,154.654755,22,de_ancient,13,29,HEROIC,T,CT,False,178536,4922
155,125,184576,Magisk,653.605591,408.358063,121.225189,22,de_ancient,13,29,Team Falcons,T,T,True,178536,6040


In [4]:
matches_maps_ids = (
    (0, 0),
    (0, 1),
    (1, 2),
    (1, 3),
    (1, 4),
    (2, 5),
    (2, 6),
    (3, 7),
    (3, 8),
    (4, 9),
    (4, 10),
    (5, 11),
    (5, 12),
    (6, 13),
    (6, 14),
    (7, 15),
    (7, 16),
    (7, 17),
    (8, 18),
    (8, 19),
    (9, 20),
    (9, 21),
    (9, 22),
    (10, 23),
    (10, 24),
    (11, 25),
    (11, 26),
    (12, 27),
    (12, 28),
    (13, 29),
    (13, 30),
    (13, 31),
    (14, 32),
    (14, 33),
    (15, 34),
    (15, 35),
    (16, 36),
    (16, 37),
    (17, 38),
    (17, 39),
    (17, 40),
    (18, 41),
    (18, 42),
    (18, 43),
    (19, 44),
    (19, 45),
    (19, 46),
    (20, 47),
    (20, 48),
    (21, 49),
    (21, 50),
    (21, 51),
    (22, 52),
    (22, 53),
    (22, 54),
    (23, 55),
    (23, 56),
    (23, 57),
    (24, 58),
    (24, 59),
    (25, 60),
    (25, 61),
    (25, 62),
    (26, 63),
    (26, 64),
    (27, 65),
    (27, 66),
    (27, 67),
    (28, 68),
    (28, 69),
    (28, 70),
)

for matchid, mapid in matches_maps_ids:
    filename = f"{matchid}_{mapid}.dem"
    if not os.path.exists(filename):
        print(f"File {filename} does not exist.")
        continue
    dem = parse_demo(filename)
    print(f"Parsed {filename} with {dem.shape[0]} rows.")

    pd.DataFrame(dem).to_csv("csv/" +filename + "_smokes.csv", index=False)




File 0_0.dem does not exist.
File 0_1.dem does not exist.
File 1_2.dem does not exist.
File 1_3.dem does not exist.
File 1_4.dem does not exist.
File 2_5.dem does not exist.
File 2_6.dem does not exist.
File 3_7.dem does not exist.
File 3_8.dem does not exist.
File 4_9.dem does not exist.
File 4_10.dem does not exist.
File 5_11.dem does not exist.
File 5_12.dem does not exist.
File 6_13.dem does not exist.
File 6_14.dem does not exist.
File 7_15.dem does not exist.
File 7_16.dem does not exist.
File 7_17.dem does not exist.
File 8_18.dem does not exist.
File 8_19.dem does not exist.
File 9_20.dem does not exist.
File 9_21.dem does not exist.
File 9_22.dem does not exist.
File 10_23.dem does not exist.
File 10_24.dem does not exist.
File 11_25.dem does not exist.
File 11_26.dem does not exist.
File 12_27.dem does not exist.
File 12_28.dem does not exist.
Parsed 13_29.dem with 157 rows.
Parsed 13_30.dem with 221 rows.
Parsed 13_31.dem with 84 rows.
Parsed 14_32.dem with 139 rows.
Parsed 

In [5]:
import pandas as pd

"""
Individual match files are named in the format:
<match_id>_<map_id>.dem.csv

Files must also be stored in the "csv" directory relative to this script.
"""

matches_maps_ids = (
    (0, 0),
    (0, 1),
    (1, 2),
    (1, 3),
    (1, 4),
    (2, 5),
    (2, 6),
    (3, 7),
    (3, 8),
    (4, 9),
    (4, 10),
    (5, 11),
    (5, 12),
    (6, 13),
    (6, 14),
    (7, 15),
    (7, 16),
    (7, 17),
    (8, 18),
    (8, 19),
    (9, 20),
    (9, 21),
    (9, 22),
    (10, 23),
    (10, 24),
    (11, 25),
    (11, 26),
    (12, 27),
    (12, 28),
    (13, 29),
    (13, 30),
    (13, 31),
    (14, 32),
    (14, 33),
    (15, 34),
    (15, 35),
    (16, 36),
    (16, 37),
    (17, 38),
    (17, 39),
    (17, 40),
    (18, 41),
    (18, 42),
    (18, 43),
    (19, 44),
    (19, 45),
    (19, 46),
    (20, 47),
    (20, 48),
    (21, 49),
    (21, 50),
    (21, 51),
    (22, 52),
    (22, 53),
    (22, 54),
    (23, 55),
    (23, 56),
    (23, 57),
    (24, 58),
    (24, 59),
    (25, 60),
    (25, 61),
    (25, 62),
    (26, 63),
    (26, 64),
    (27, 65),
    (27, 66),
    (27, 67),
    (28, 68),
    (28, 69),
    (28, 70),
)

df_composite = pd.DataFrame()

num_rounds = 0

for match_id, map_id in matches_maps_ids:
    file_name = f"csv/{match_id:02d}_{map_id:02d}.dem_smokes.csv"

    try:
        df_match = pd.read_csv(file_name)
        df_match["round_id"] = df_match["round_number"] + num_rounds - 1  # 0 indexing

        round_ids = list(map(int, df_match["round_id"].unique()))
        map_rounds = len(round_ids)
        
        print(f"Parsing {file_name}: {map_rounds} rounds")
        
        df_composite = pd.concat([df_composite, df_match])
        num_rounds += map_rounds
    except FileNotFoundError:
        print(f"{file_name} not found. Skipping.")
        continue
    except Exception as e:
        print(f"File error: {e}")
        continue

df_composite.to_csv("IEM_DALLAS_2025_SMOKES.csv", index=False)


csv/00_00.dem_smokes.csv not found. Skipping.
csv/00_01.dem_smokes.csv not found. Skipping.
csv/01_02.dem_smokes.csv not found. Skipping.
csv/01_03.dem_smokes.csv not found. Skipping.
csv/01_04.dem_smokes.csv not found. Skipping.
csv/02_05.dem_smokes.csv not found. Skipping.
csv/02_06.dem_smokes.csv not found. Skipping.
csv/03_07.dem_smokes.csv not found. Skipping.
csv/03_08.dem_smokes.csv not found. Skipping.
csv/04_09.dem_smokes.csv not found. Skipping.
csv/04_10.dem_smokes.csv not found. Skipping.
csv/05_11.dem_smokes.csv not found. Skipping.
csv/05_12.dem_smokes.csv not found. Skipping.
csv/06_13.dem_smokes.csv not found. Skipping.
csv/06_14.dem_smokes.csv not found. Skipping.
csv/07_15.dem_smokes.csv not found. Skipping.
csv/07_16.dem_smokes.csv not found. Skipping.
csv/07_17.dem_smokes.csv not found. Skipping.
csv/08_18.dem_smokes.csv not found. Skipping.
csv/08_19.dem_smokes.csv not found. Skipping.
csv/09_20.dem_smokes.csv not found. Skipping.
csv/09_21.dem_smokes.csv not found